---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [24]:
# housing=pd.read_csv('City_Zhvi_AllHomes.csv')
# housing['State'].replace(states,inplace=True)
# housing_trans=housing.T
# housing_trans=housing_trans.loc['2000-01':]

# housing_trans.index=pd.to_datetime(housing_trans.index)
# resample=housing_trans.resample('Q')
# resample=resample.apply(np.mean,axis=0)
# resample=resample.applymap(np.around)
# year=pd.Series(resample.index.year.astype('str'))+'q'
# resample.index=year.str.cat(resample.index.quarter.astype('str'))
# resample=resample.T
# resample['RegionName']=housing['RegionName']
# resample['State']=housing['State']
# resample.set_index(['State','RegionName'])

In [11]:
# university_df=get_list_of_university_towns()
# housing_df=convert_housing_data_to_quarters()
# tmp=university_df.set_index(['State','Region Name'])
# university_housing=housing_df[housing_df.index.isin(tmp.index)]
# nonuniversity_housing=housing_df[~housing_df.index.isin(tmp.index)]
# start=get_recession_start()
# bottom=get_recession_bottom()
# if results.pvalue<0.01:
#     different=True
# else:
#     different=False
# if university_housing['diff'].mean()<nonuniversity_housing['diff'].mean():
#     better='university town'
# else:
#     better='nonuniversity town'


In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df1=pd.read_table('university_towns.txt',header=None)
    state_df=df1[df1.iloc[:,0].str.contains('edit')]
    state_df=state_df.iloc[:,0].str.replace('\[.*\]','')
    town_df=df1[np.logical_not(df1.iloc[:,0].str.contains('edit'))]
    town_df=town_df.iloc[:,0].str.replace('\s\(.*','')
    diff_array=np.append(np.diff(state_df.index),[np.Inf])
    outlist=[]
    for i,j in zip(state_df.index,diff_array):
        for k in town_df.index:
            if i<k and i+j>k:
                outlist.append([state_df[i],town_df[k]])
            
    ans=pd.DataFrame(outlist).rename(columns={0:'State',1:'RegionName'})
    return ans

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',skiprows=7).iloc[:,[4,6]].rename(columns={'Unnamed: 4':'quarter',
                                                                         'Unnamed: 6':'GDP in current dollar'})
    df=df[df['quarter'].str.slice(0,4).astype('int')>=2000]
    df['change']=df.iloc[:,1].diff()
    neg_q=df[df['change']<0].index
    neg_q=np.append(neg_q,np.Inf).astype('int')
    out2=[]
    out=[]
    tmp=[]
    n=1
    for i in range(len(neg_q)-1):
        tmp.append(neg_q[i])
        if neg_q[i+1]-neg_q[i]<=1:
            None
        else:
            out.append(tmp)
            tmp=[]
    for i in out:
        if len(i)>=2:
            out2.append(i)
    start=out2[0][0]
    
    return df.loc[start][0]


In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',skiprows=7).iloc[:,[4,6]].rename(columns={'Unnamed: 4':'quarter',
                                                                     'Unnamed: 6':'GDP in current dollar'})
    df=df[df['quarter'].str.slice(0,4).astype('int')>=2000]
    df['change']=df.iloc[:,1].diff()
    recession_ind=(df[df['quarter']==get_recession_start()].index)[0]
    df_after=df[df.index>recession_ind]
    for i in range(1,len(df_after)):
        if df_after.iloc[i,2]>0 and df_after.iloc[i-1,2]>0:
            end=df_after.iloc[i,0]
            break
    return end


In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    df=pd.read_excel('gdplev.xls',skiprows=7).iloc[:,[4,6]].rename(columns={'Unnamed: 4':'quarter',
                                                                         'Unnamed: 6':'GDP in current dollar'})
    df=df[df['quarter'].str.slice(0,4).astype('int')>=2000]
    df['change']=df.iloc[:,1].diff()
    end=get_recession_end()
    start_ind=(df[df['quarter']==get_recession_start()].index)[0]
    end_ind=(df[df['quarter']==end].index)[0]
    recession=df[np.logical_and(df.index>=start_ind,df.index<=end_ind)]
    bottom=recession.loc[recession['GDP in current dollar'].argmin()][0]
    return bottom


In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing=pd.read_csv('City_Zhvi_AllHomes.csv')
    housing['State'].replace(states,inplace=True)
    housing_trans=housing.T
    housing_trans=housing_trans.loc['2000-01':]

    housing_trans.index=pd.to_datetime(housing_trans.index)
    resample=housing_trans.resample('Q')
    resample=resample.apply(np.mean,axis=0)
    resample=resample.applymap(np.around)
    year=pd.Series(resample.index.year.astype('str'))+'q'
    resample.index=year.str.cat(resample.index.quarter.astype('str'))
    resample=resample.T
    resample['RegionName']=housing['RegionName']
    resample['State']=housing['State']
    ans=resample.set_index(['State','RegionName'])
    return ans


In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    university_df=get_list_of_university_towns()
    housing_df=convert_housing_data_to_quarters()
    tmp=university_df.set_index(['State','RegionName'])

    university_housing=housing_df[housing_df.index.isin(tmp.index)]
    nonuniversity_housing=housing_df[~housing_df.index.isin(tmp.index)]
    start=get_recession_start()
    bottom=get_recession_bottom()
    university_housing['ratio']=np.divide(university_housing[bottom],university_housing[start])
    nonuniversity_housing['ratio']=np.divide(nonuniversity_housing[bottom],nonuniversity_housing[start])
    results=ttest_ind(university_housing['ratio'],nonuniversity_housing['ratio'],nan_policy='omit')
    if results.pvalue<0.01:
        different=True
    else:
        different=False
    if university_housing['ratio'].mean()>nonuniversity_housing['ratio'].mean():
        better='university town'
    else:
        better='nonuniversity town'
    ans=(different,results.pvalue,better)
    
    return ans